In [1]:
import numpy as np
import scipy as sp
from scipy.sparse.linalg import LinearOperator

import MathFunctions as MF
import MPSOperators as MO
import SingleVUMPS as SV

In [2]:
# Eq. (12)
def Calc_hr(AR,h):
    ARAR = np.tensordot(AR,AR,([2],[0]))
    ARARh = np.tensordot(ARAR,h,([1,2],[0,1]))
    hr = np.tensordot(ARARh,np.conj(ARAR),([2,3,1],[1,2,3]))
    return hr

# Eq. (12)
def Calc_hl(AL,h):
    ALAL = np.tensordot(AL,AL,([2],[0]))
    ALALh = np.tensordot(ALAL,h,([1,2],[0,1]))
    hl = np.tensordot(ALALh,np.conj(ALAL),([0,2,3],[0,1,2]))
    return hl

# Calculate the Energy by using hr
def Calc_right_energy(AR,h,dtype):
    M = AR.shape[0]
    hr = Calc_hr(AR,h)
    _,vecL = MF.RightEigs(AR.transpose(2,1,0),dtype)
    VL = vecL.reshape(M,M)
    VL /= np.trace(VL)
    energy = np.inner(hr.reshape(M*M),VL.reshape(M*M))
    return energy

# Calculate the Energy by using hl
def Calc_left_energy(AL,h,dtype):
    M = AL.shape[0]
    hl = Calc_hl(AL,h)
    _,vecR = MF.RightEigs(AL,dtype)
    VR = vecR.reshape(M,M)
    VR /= np.trace(VR)
    energy = np.inner(hl.reshape(M*M),VR.reshape(M*M))
    return energy

In [3]:
def Calc_HR(AR,HR,h,dtype,tol=1e-17):
    M = AR.shape[0]
    hr = Calc_hr(AR,h)
    er = Calc_right_energy(AR,h,dtype)
    hr_tilde = hr - er * np.eye(M,M)
    #HR_new = Calc_HR_PowerMethod(AR,hr_tilde)
    #HR_new = Calc_HR_Inverse(AR,hr_tilde)
    HR_new = Simple_Calc_HR_Linear(AR,HR,hr_tilde,dtype,tol)
    #HR_new = Calc_HR_Linear(AR,HR,hr_tilde,dtype,tol)
    return HR_new,er

def Calc_HL(AL,HL,h,dtype,tol=1e-17):
    M = AL.shape[0]
    hl = Calc_hl(AL,h)
    el = Calc_left_energy(AL,h,dtype)
    hl_tilde = hl - el * np.eye(M,M)
    #HL_new = Calc_HL_PowerMethod(AL,hl_tilde)
    #HL_new = Calc_HL_Inverse(AL,hl_tilde)
    HL_new = Simple_Calc_HL_Linear(AL,HL,hl_tilde,dtype,tol)
    return HL_new,el

# Eq. (15)
def Simple_Calc_HR_Linear(AR,HR,hr,dtype,tol):
    M = AR.shape[0]
    TR = np.einsum("asc,bsd -> abcd",AR,np.conj(AR)).reshape(M*M,M*M)
    _,vecL = MF.RightEigs(AR.transpose(2,1,0),dtype)
    VL = vecL.reshape(M,M)
    VL /= np.trace(VL)
    #_,vecL = MF.RightEigs(AR,dtype)
    #VL = vecL.reshape(M,M)
    #VL /= np.trace(VL)
    P = np.einsum("ab,cd -> abcd",np.eye(M),VL).reshape(M*M,M*M)
    PL = np.eye(M*M) - P
    R = np.einsum("ab,b -> a",PL,hr.reshape(M*M))
    T = np.eye(M*M) - TR + P
    HR,_ = sp.sparse.linalg.bicgstab(T,R,atol=tol,tol=tol,x0=HR.reshape(M*M))
    return HR.reshape(M,M)

# Eq. (15)
def Simple_Calc_HL_Linear(AL,HL,hl,dtype,tol):
    M = AL.shape[0]
    TL = np.einsum("asc,bsd -> abcd",AL,np.conj(AL)).reshape(M*M,M*M)
    _,vecR = MF.RightEigs(AL,dtype)
    VR = vecR.reshape(M,M)
    VR /= np.trace(VR)
    #_,vecR = MF.RightEigs(AL.transpose(2,1,0),dtype)
    #VR = vecR.reshape(M,M)
    #VR /= np.trace(VR)
    P = np.einsum("ab,cd -> abcd",np.eye(M),VR).reshape(M*M,M*M)
    PR = np.eye(M*M) - P
    R = np.einsum("ab,b -> a",PR,hl.reshape(M*M))
    T = np.eye(M*M) - TL.T + P
    HL,_ = sp.sparse.linalg.bicgstab(T,R,atol=tol,tol=tol,x0=HL.reshape(M*M))
    return HL.reshape(M,M)

def Calc_HR_Linear(AR,HR,hr_tilde,dtype,tol):
    M = AR.shape[0]
    _,vecL = MF.RightEigs(AR,dtype)
    VL = vecL.reshape(M,M)
    VL /= np.trace(VL)
    T_ope = LinearSolver_ope(AR,VL,dtype)
    HR,_ = sp.sparse.linalg.bicgstab(T_ope,hr_tilde.reshape(M*M),tol=0,atol=tol,x0=HL.reshape(M*M))
    return HR.reshape(M,M)
    
class LinearSolver_ope(sp.sparse.linalg.LinearOperator):
    def __init__(self,A,L,dtype):
        self.A = A
        self.L = L
        self.M = A.shape[0]
        self.D = A.shape[1]
        self.shape = [self.M * self.M, self.M * self.M]
        self.dtype = dtype
    def _matvec(self,Rvec):
        R = Rvec.reshape(self.M,self.M)
        ARR = np.tensordot(self.A,R,([2],[0]))
        ARRA = np.tensordot(ARR,np.conj(self.A),([1,2],[1,2]))
        R -= ARRA - np.eye(self.M) * np.inner(self.L.reshape(self.M*self.M),Rvec)
        return R
    def _rmatvec(self,Rvec):
        R = Rvec.reshape(self.M,self.M)
        AT = self.A.transpose(2,1,0)
        ARR = np.tensordot(np.conj(AT),R,([2],[0]))
        ARRA = np.tensordot(ARR,AT,([1,2],[1,2]))
        R -= ARRA - np.conj(self.L) * np.trace(R)
        return R

In [4]:
D = 2; M = 10
#dtype = np.dtype("float"); A = np.random.rand(M,D,M)
dtype = np.dtype("complex"); A = np.random.rand(M,D,M) + 1j * np.random.rand(M,D,M); 
Sx,Sy,Sz,Su,Sd = MF.Spin(D)
h = - ( np.kron(Sx,Sx) + np.kron(Sy,Sy) - np.kron(Sz,Sz) ).real.reshape(D,D,D,D)
#h =  ( np.kron(Sx,Sx) + np.kron(Sy,Sy) + np.kron(Sz,Sz) ).real.reshape(D,D,D,D)
AC,C,AR,AL = MO.MixedCanonicalForm(A,dtype)
HR = np.random.rand(M,M); HL = np.random.rand(M,M) # initial HR and HL
for i in range (50):
    HR,er = Calc_HR(AR,HR,h,dtype)
    HL,el = Calc_HL(AL,HL,h,dtype)
    AC = SV.Next_AC(AC,AR,AL,HR,HL,h,dtype)
    C = SV.Next_C(C,AR,AL,HR,HL,h,dtype)
    AR = SV.Next_AR_SVD(AC,C)
    #AR = SV.Next_AR_PolarDecomposition(AC,C)
    AL = SV.Next_AL_SVD(AC,C)
    #AL = SV.Next_AL_PolarDecomposition(AC,C)
    B = SV.Calc_B(AC,C,AR,AL)
    print (er,el,B)

(-0.25274590078015874-3.238715177121543e-17j) (-0.25274590078015885-4.470978708787099e-17j) 0.28827500746867213
(0.21724347762572332+3.9898639947466563e-17j) (0.2253954608989333+1.244664094013359e-16j) 0.2625107722391492
(-0.4109583803654607-1.7867651802561113e-16j) (-0.42519459143621186-7.979727989493313e-17j) 0.678314381716673
(-0.04445149720774048-4.5102810375396984e-17j) (0.07662001195964918+5.377642775528102e-17j) 0.19467579728385767
(-0.42516844742561055+7.979727989493313e-17j) (-0.41063624677994304+1.6132928326584306e-16j) 0.29102785713345164
(0.22927208518897493+1.9081958235744878e-17j) (0.18174240566602393+1.0104764247564901e-16j) 0.5682381655803262
(-0.39913434238731643-1.2923689896027213e-16j) (-0.40948873392100454+6.938893903907228e-18j) 0.3408517518664039
(0.13277387529335435-7.632783294297951e-17j) (0.18381487442453215-4.163336342344337e-17j) 0.3702464972046428
(-0.4218991013356324+1.0408340855860843e-16j) (-0.42336618087294015-1.4311468676808659e-16j) 1.1328230110187707
